# Baseline for Ubiquant Market Prediction using ANN

**Thanks to notebook contributors for make baseline for my notebook**
- https://www.kaggle.com/code/sohier/competition-api-detailed-introduction/notebook
- https://www.kaggle.com/code/pythonash/end-to-end-simple-and-powerful-dnn-with-leakyrelu
- https://www.kaggle.com/code/robikscube/fast-data-loading-and-low-mem-with-parquet-files

First import libraries and dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.keras as keras
import tensorflow as tf
import cudf
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
df = df.astype("float16")

# Preprocess dataset

Because of it's baseline I use only f_n for features for X_train.

In [ ]:
index_col = df.drop(["row_id", "time_id", "target", "investment_id"], axis=1).columns
X_train = df[index_col]

In [ ]:
y_train = df["target"]

In [ ]:
X_train, y_train, X_test, y_test = X_train[0:2122479], y_train[0:2122479], X_train[2122479:], y_train[2122479:]

# Making model

I make simple model using Dense layer and BatchNormalizaion without hyper parameter tuning.

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_mse", patience=4, mode="min")
mc = keras.callbacks.ModelCheckpoint("best_model.h5", monitor="val_mse", save_best_only=True, mode="min", verbose=1)

In [ ]:
Input = keras.layers.Input(shape=X_train.shape[1])
x = keras.layers.Dense(128, activation="relu")(Input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(16, activation="relu")(x)
x = x = keras.layers.BatchNormalization()(x)
Output = keras.layers.Dense(1, activation="linear")(x)

model = keras.models.Model(inputs = Input, outputs = Output)

In [ ]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[es, mc])

# Submission

In [ ]:
import ubiquant
model = tf.keras.models.load_model('best_model.h5')
env = ubiquant.make_env()   
iter_test = env.iter_test()    
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df["target"] = model.predict(test_df[index_col])
    env.predict(sample_prediction_df)